# Explainability Agent

zu klären:
- lime: as_list
- shap: https://github.com/slundberg/shap/issues/632
- logit: fällt wahrscheinlich raus / zu schlecht, ada wahrscheinlich auch
- ebm Output = Feature importance?
- verschiedene Modelle für activity & usage? --> hinterher die pickles zusammenwerfen?

Goal: nur model, welches wir am Ende nehmen, erklären

Overall: nach activity agent bestes model (accuracy) hier rein, kommt aus Kai's Liste für verschiedene Households --> feature importance als input für recommendation

Recommendation verlängern: Unsere Empfehlung sprechen wir aus, weil wir denken, dass du heute den tumble dryer benutzt, weil 3 höchste feature importances. 

Features in Lesbares umwandeln, z.B. activity_lag_1 --> Du warst in den letzten 24 Stunden zuhause.
--> Wenn aber alle 3 letzten Features activity_lags sind, nehmen wir die 4. feature importance statt der 2. und 3. oder so

Plot: 
- shap: feature importance: image = True
- lime: 

Recommendation an sich: 
- relative savings aus Alonas Paper?

In [ ]:
class Explainability_Agent:
    def __init__(self):
        ### Agents as input ?
        ### get feature importances 
        